In [1]:
import papermill as pm
import multiprocessing as mp
from datetime import datetime
import traceback

mp.set_start_method('spawn')

In [2]:
number_simulations = 500
number_samples = int(15e6)
pop_size = int(5e8)
rate = 2
entropy = 42

In [3]:
simulation_numbers = list(range(1, number_simulations+1))

In [4]:
prettify = lambda integer: str(integer).zfill(len(str(number_simulations)))

In [5]:
input_notebook_names = ['notebook_template.ipynb' for simulation_number in simulation_numbers]

output_notebook_names = ['notebook_logs/results_log.{}.ag.ipynb'.format(prettify(simulation_number))
                        for simulation_number in simulation_numbers]

parameter_dicts = [{'number_simulations':number_simulations, 'simulation_number':simulation_number,
                   'number_samples':number_samples, 'pop_size':pop_size, 'rate':rate, 'entropy':entropy}
                  for simulation_number in simulation_numbers]

In [6]:
parallel_input = [[input_notebook_name, output_notebook_name, parameter_dict]
                 for input_notebook_name, output_notebook_name, parameter_dict
                 in zip(input_notebook_names, output_notebook_names, parameter_dicts)]

In [7]:
%%time
try:
    pool = mp.Pool(40)
    pool.starmap(pm.execute_notebook, parallel_input)
    pool.close()
    pool.join()
# These errors usually don't seem to actually matter. All of the results still get produced
# and I'm not sure what the cause is, and I don't really care. Maybe it's the %%time.
# I just want some record of how slow this is, is that so much to ask? yes I get that a
# Jupyter cell magic probably is not the correct way to do it, but I'm a small person who
# wants to stick what they're familiar and comfortable with sometimes, OK? Sorry...
except RuntimeError as error_message:
    timestamp_filename = datetime.now().strftime('%m-%d-%Y.at.%H_%M_%S.log')
    with open('error_logs/{}'.format(timestamp_filename), 'a') as file_pointer:
        file_pointer.write(str(error_message))
        file_pointer.write(traceback.format_exc())

CPU times: user 1.13 s, sys: 2.34 s, total: 3.46 s
Wall time: 1h 15min 55s
